In [4]:
require 'image'
npy4th = require 'npy4th'
require 'data_aflw';
require 'cunn'
require 'cudnn'
require 'nn';
npy4th=require 'npy4th';
require 'torchx';



function plotPoint(im,point,pointSize,color_curr)
    if x>im:size(3) or y>im:size(2) then
        return im;
    end
    local point=torch.Tensor(point);
    local starts=torch.round(point-pointSize/2);
    local ends=torch.round(point+pointSize/2);

    for x_curr=math.max(1,starts[1]),math.min(im:size(3),ends[1]) do
        for y_curr=math.max(1,starts[2]),math.min(im:size(2),ends[2]) do
            for idx_rgb=1,3 do
                im[idx_rgb][y_curr][x_curr]=color_curr[idx_rgb]/255;            
            end
        end
    end

    return im;

end


function drawKeyPoints(im,keypoints,scale,colors,pointSize)
    assert (#keypoints:size()==2);
    assert (keypoints:size(1)==2);
    assert (im:size(1)==3);
    
    if not pointSize then
        pointSize=math.max(torch.round(math.min(im:size(2),im:size(3))*0.02),1);
    end
    
    if not colors then
        colors={{255,0,0}};
    end

    if torch.max(im)>1 then
        im=im/255;
    end

    if scale~=nil then
        assert (#scale==2);
        keypoints=keypoints-scale[1];
        keypoints=torch.div(keypoints,scale[2]-scale[1]);
        keypoints[{1,{}}]=keypoints[{1,{}}]*im:size(2);
        keypoints[{2,{}}]=keypoints[{2,{}}]*im:size(3);
    end

    for label_idx=1,keypoints:size(2) do
        x=keypoints[2][label_idx];
        y=keypoints[1][label_idx];
        local color_curr=colors[math.min(#colors,label_idx)];
        im=plotPoint(im,{x,y},pointSize,color_curr);
    end

    return im;
end


meta_debug_dir='/home/SSD3/maheen-data/temp/debug_aflw_40_vanilla_selected_drop_smooth_vanilla';

out_dir = paths.concat(meta_debug_dir,'model_th');
paths.mkdir(out_dir);
model_file=paths.concat(out_dir,'fc2_100.dat');
model_file=paths.concat(out_dir,'/home/SSD3/maheen-data/horse_project/vanilla/vanilla_original.dat')
data_file='/home/SSD3/maheen-data/horse_project/data_check/face/pairs.txt';
mean_im_path='/home/laoreja/finetune-deep-landmark/dataset/train/aflw_40_mean.png'
std_im_path='/home/laoreja/finetune-deep-landmark/dataset/train/aflw_40_std.png'


local data_params={file_path_horse=data_file,
					batch_size=64,
					mean_file=mean_im_path,
					std_file=std_im_path};

td=data(data_params);
td:getTrainingData();
local idx_test=1;
local im_curr=td.training_set_horse.data[idx_test];
print (im_curr:size(),torch.min(im_curr),torch.max(im_curr))

print (td.training_set_horse.data:size())
print (td.training_set_horse.label:size())

model=torch.load(model_file);
model:evaluate();
model:cuda();
td.training_set_horse.data=td.training_set_horse.data:cuda();
results=model:forward(td.training_set_horse.data);
print (results:size());

td.training_set_horse.data=td.training_set_horse.data:double();

colors={{255,0,0},{255,255,0},{0,0,255},{0,255,0},{255,0,255}}

for idx_test=1,results:size(1) do
	local im_curr=td.training_set_horse.data[idx_test];
    print (torch.min(im_curr),torch.max(im_curr))
	-- im_curr=torch.cmul(im_curr,td.std_im);
	-- im_curr=im_curr+td.mean_im;
-- 	local im_curr=image.load(td.training_set_horse.input[idx_test]);
	
-- 	local labels_gt=td.training_set_horse.label[idx_test]
-- 	labels_gt=labels_gt[{{},{1,2}}]:t();
-- 	print (labels_gt);

-- 	local labels_pred=results[idx_test];
-- 	-- print (labels_pred)
-- 	labels_pred=labels_pred:resize(5,2);
	
-- 	local t=labels_pred[{{},1}]:clone();
-- 	labels_pred[{{},1}]=labels_pred[{{},2}];
-- 	labels_pred[{{},2}]=t;

-- 	labels_pred=labels_pred:t();

-- 	local im_pred=drawKeyPoints(im_curr:clone(),labels_pred,{-0.5,0.5},colors);

-- 	itorch.image(im_pred)
    
--     im_curr=drawKeyPoints(im_curr,labels_gt,{-0.5,0.5},colors);
-- 	itorch.image(im_curr)


	-- break
end


-- print (td.training_set_horse.label[1])



10000	


  3
 40
 40
[torch.LongStorage of size 3]

-1	1	
 64
  3
 40
 40
[torch.LongStorage of size 4]

 64
  5
  3
[torch.LongStorage of size 3]



 64
 10
[torch.LongStorage of size 2]



-1	1	
-1	1	
-1	2	
-2	1	
-1	1	
-1	1	
-1	2	
-1	2	
-1	1	
-1	1	
-1	1	
-1	2	
-1	2	
-1	1	


-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	2	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-2	0	
-1	2	
-1	1	
-1	1	
-1	0	
-2	2	
-1	1	
-1	2	
0	1	
-1	1	
-1	1	
-1	2	
-1	1	
-1	1	
-1	1	
-1	1	
-1	1	
-1	2	
-1	2	
-1	1	
-1	1	
-2	1	
-1	1	
-1	1	
-1	2	
-1	1	
-1	2	
-1	1	
-1	1	
-1	1	
-1	2	
-1	0	
-1	2	
0	1	


In [1]:
require 'nn';
require 'loadcaffe';


local deploy_file='/home/maheenrashid/Downloads/horses/torch//vanilla_deploy.prototxt';
local caffe_model_file='/home/laoreja/data/VanillaCNN/ZOO/vanillaCNN.caffemodel'
local out_dir='/home/SSD3/maheen-data/horse_project/vanilla/'
paths.mkdir(out_dir);
local out_file=paths.concat(out_dir,'vanilla_original.dat');

local model = loadcaffe.load(deploy_file,caffe_model_file, 'nn')
print (model);

model:insert(nn.Tanh(),10);
model:insert(nn.Abs(),11);

model:insert(nn.Tanh(),8);
model:insert(nn.Abs(),9);

model:insert(nn.Tanh(),6);
model:insert(nn.Abs(),7);

model:insert(nn.Tanh(),4);
model:insert(nn.Abs(),5);

model:insert(nn.Tanh(),2);
model:insert(nn.Abs(),3);

print (model);

torch.save(out_file,model);


Successfully loaded /home/laoreja/data/VanillaCNN/ZOO/vanillaCNN.caffemodel


Conv1: 16 3 5 5
Conv2: 48 16 3 3
Conv3: 64 48 3 3


Conv4: 64 64 2 2
Dense1: 1 1 576 100
Dense2: 1 1 100 10
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> output]
  (1): nn.SpatialConvolution(3 -> 16, 5x5, 1,1, 2,2)
  (2): nn.SpatialMaxPooling(2x2, 2,2)
  (3): nn.SpatialConvolution(16 -> 48, 3x3, 1,1, 1,1)
  (4): nn.SpatialMaxPooling(2x2, 2,2)
  (5): nn.SpatialConvolution(48 -> 64, 3x3)
  (6): nn.SpatialMaxPooling(3x3, 2,2)
  (7): nn.SpatialConvolution(64 -> 64, 2x2)
  (8): nn.View(-1)
  (9): nn.Linear(576 -> 100)
  (10): nn.Linear(100 -> 10)
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.SpatialConvolution(3 -> 16, 5x5, 1,1, 2,2)
        {
          dH : 1
          dW : 1
          nInputPlane : 3
          output : DoubleTensor - empty
          kH : 5
          name : Conv1
          gradBias : DoubleTensor - size: 16
          padH : 2
          bias : DoubleTensor - size: 16
          weight : DoubleTensor - size: 16x3x5x5
          _type : torch.

    padW : 2
          nOutputPlane : 16
          kW : 5
          gradInput : DoubleTensor - empty
        }
      2 : 
        nn.SpatialMaxPooling(2x2, 2,2)
        {
          dH : 2
          dW : 2
          kW : 2
          gradInput : DoubleTensor - empty
          indices : DoubleTensor - empty
          name : Pool1
          _type : torch.DoubleTensor
          padH : 0
          ceil_mode : true
          output : DoubleTensor - empty
          kH : 2
          padW : 0
        }
      3 : 
        nn.SpatialConvolution(16 -> 48, 3x3, 1,1, 1,1)
        {
          dH : 1
          dW : 1
          nInputPlane : 16
          output : DoubleTensor - empty
          kH : 3
          name : Conv2
          gradBias : DoubleTensor - size: 48
          padH : 1
          bias : DoubleTensor - size: 48
          weight : DoubleTensor - size: 48x16x3x3
          _type : torch.DoubleTensor
          gradWeight : DoubleTensor - size: 48x16x3x3
          padW : 1
          nOutputPla

       dH : 2
          dW : 2
          kW : 2
          gradInput : DoubleTensor - empty
          indices : DoubleTensor - empty
          name : Pool2
          _type : torch.DoubleTensor
          padH : 0
          ceil_mode : true
          output : DoubleTensor - empty
          kH : 2
          padW : 0
        }
      5 : 
        nn.SpatialConvolution(48 -> 64, 3x3)
        {
          dH : 1
          dW : 1
          nInputPlane : 48
          output : DoubleTensor - empty
          kH : 3
          name : Conv3
          gradBias : DoubleTensor - size: 64
          padH : 0
          bias : DoubleTensor - size: 64
          weight : DoubleTensor - size: 64x48x3x3
          _type : torch.DoubleTensor
          gradWeight : DoubleTensor - size: 64x48x3x3
          padW : 0
          nOutputPlane : 64
          kW : 3
          gradInput : DoubleTensor - empty
        }
      6 : 
        nn.SpatialMaxPooling(3x3, 2,2)
        {
          dH : 2
          dW : 2
          kW

...

   bias : DoubleTensor - size: 10
          gradWeight : DoubleTensor - size: 10x100
        }
    }
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
}
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> output]
  (1): nn.SpatialConvolution(3 -> 16, 5x5, 1,1, 2,2)
  (2): nn.Tanh
  (3): nn.Abs
  (4): nn.SpatialMaxPooling(2x2, 2,2)
  (5): nn.SpatialConvolution(16 -> 48, 3x3, 1,1, 1,1)
  (6): nn.Tanh
  (7): nn.Abs
  (8): nn.SpatialMaxPooling(2x2, 2,2)
  (9): nn.SpatialConvolution(48 -> 64, 3x3)
  (10): nn.Tanh
  (11): nn.Abs
  (12): nn.SpatialMaxPooling(3x3, 2,2)
  (13): nn.SpatialConvolution(64 -> 64, 2x2)
  (14): nn.Tanh
  (15): nn.Abs
  (16): nn.View(-1)
  (17): nn.Linear(576 -> 100)
  (18): nn.Tanh
  (19): nn.Abs
  (20): nn.Linear(100 -> 10)
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.SpatialConvolution(3 -> 16, 

          padH : 0
          ceil_mode : true
          output : DoubleTensor - empty
          kH : 2
          padW : 0
        }
      5 : 
        nn.SpatialConvolution(16 -> 48, 3x3, 1,1, 1,1)
        {
          dH : 1
          dW : 1
          nInputPlane : 16
          output : DoubleTensor - empty
          kH : 3
          name : Conv2
          gradBias : DoubleTensor - size: 48
          padH : 1
          bias : DoubleTensor - size: 48
          weight : DoubleTensor - size: 48x16x3x3
          _type : torch.DoubleTensor
          gradWeight : DoubleTensor - size: 48x16x3x3
          padW : 1
          nOutputPlane : 48
          kW : 3
          gradInput : DoubleTensor - empty
        }
      6 : 
        nn.Tanh
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
      7 : 
        nn.Abs
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
     

      nInputPlane : 48
          output : DoubleTensor - empty
          kH : 3
          name : Conv3
          gradBias : DoubleTensor - size: 64
          padH : 0
          bias : DoubleTensor - size: 64
          weight : DoubleTensor - size: 64x48x3x3
          _type : torch.DoubleTensor
          gradWeight : DoubleTensor - size: 64x48x3x3
          padW : 0
          nOutputPlane : 64
          kW : 3
          gradInput : DoubleTensor - empty
        }
      10 : 
        nn.Tanh
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
      11 : 
        nn.Abs
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
      12 : 
        nn.SpatialMaxPooling(3x3, 2,2)
        {
          dH : 2
          dW : 2
          kW : 3
          gradInput : DoubleTensor - empty
          indices : DoubleTensor - empty
   

          bias : DoubleTensor - size: 64
          weight : DoubleTensor - size: 64x64x2x2
          _type : torch.DoubleTensor
          gradWeight : DoubleTensor - size: 64x64x2x2
          padW : 0
          nOutputPlane : 64
          kW : 2
          gradInput : DoubleTensor - empty
        }
      14 : 
        nn.Tanh
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
      15 : 
        nn.Abs
        {
          gradInput : DoubleTensor - empty
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
        }
      16 : 
        nn.View(-1)
        {
          name : torch_view
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          size : LongStorage - size: 1
          numInputDims : 3
          numElements : 1
        }
      17 : 
        nn.Linear(576 -> 100)
        {
          gradBias : D

        _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          name : Dense2
          bias : DoubleTensor - size: 10
          gradWeight : DoubleTensor - size: 10x100
        }
    }
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
}
